In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader

from AI.pytorch.tutorial.src import nn_loss

In [2]:
inputs = torch.tensor([1, 2, 3], dtype=torch.float32)
targets = torch.tensor([1, 2, 5], dtype=torch.float32)

inputs = torch.reshape(inputs, (1, 1, 1, 3))
targets = targets.reshape(1, 1, 1, 3)
print(inputs)
print(targets)

tensor([[[[1., 2., 3.]]]])
tensor([[[[1., 2., 5.]]]])


In [3]:
# 平均差
loss =  nn.L1Loss()
result = loss(inputs, targets)
print(result)
print(result.shape)

tensor(0.6667)
torch.Size([])


In [4]:
# ((targets-inputs)^2).mean()
loss_mse = nn.MSELoss()
result_mse = loss_mse(inputs, targets)
print(result_mse)
print(result_mse.shape)

tensor(1.3333)
torch.Size([])


第一個標签的概率為0.2

-0.2 + ln(exp(0.1)+exp(0.2)+exp(0.3))

In [5]:
x = torch.tensor([0.1, 0.2, 0.3])
y = torch.tensor([1], dtype=torch.long) # label
x = torch.reshape(x, (1, 3))
print(x, y)
loss_cross = nn.CrossEntropyLoss()
result_cross = loss_cross(x, y)
print(result_cross)

tensor([[0.1000, 0.2000, 0.3000]]) tensor([1])
tensor(1.1019)


In [6]:
dataset = torchvision.datasets.CIFAR10(root='./dataset', train=False, download=True, transform=torchvision.transforms.ToTensor())
print(type(dataset))
dataloader = DataLoader(dataset, batch_size=64)
print(type(dataloader))

Files already downloaded and verified
<class 'torchvision.datasets.cifar.CIFAR10'>
<class 'torch.utils.data.dataloader.DataLoader'>


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.maxpool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 32, 5, padding=2)
        self.maxpool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.maxpool3 = nn.MaxPool2d(2)

        self.flatten = nn.Flatten()
        # 32/2/2/2 = 4
        # 64 是因為 conv3的out_channels是64
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)

        x = self.flatten(x) # nn.Flatten()类的flatten功能start_dim默認為1
        # x = torch.flatten(x, 1) # torch.flatten功能的start_dim默認為0
        x = self.fc1(x)
        x = self.fc2(x)
        return x

training

In [13]:
loss = nn.CrossEntropyLoss()
cnn = CNN()
optim = torch.optim.SGD(cnn.parameters(), lr=0.01) # lr學習速率
for epoch in range(20): # 進行多輪學習
    running_loss = 0.0 # 總體誤差和
    for data in dataloader:
        imgs, targets = data
        imgs = (imgs.float() - imgs.float().mean()) / imgs.float().std()
        output = cnn(imgs)

        result_loss = loss(output, targets)

        optim.zero_grad() # 清空梯度, 防止之前的梯度痕跡
        result_loss.backward() # 反向傳播,計算每個參數的梯度
        optim.step() # 對每個參數調優
        # print(result_loss)
        running_loss += result_loss
    print(running_loss)



tensor(336.9356, grad_fn=<AddBackward0>)
tensor(304.2004, grad_fn=<AddBackward0>)
tensor(286.0496, grad_fn=<AddBackward0>)
tensor(267.0109, grad_fn=<AddBackward0>)
tensor(252.5983, grad_fn=<AddBackward0>)
tensor(241.2928, grad_fn=<AddBackward0>)
tensor(231.9112, grad_fn=<AddBackward0>)
tensor(223.7349, grad_fn=<AddBackward0>)
tensor(216.3551, grad_fn=<AddBackward0>)
tensor(209.5089, grad_fn=<AddBackward0>)
tensor(203.1064, grad_fn=<AddBackward0>)
tensor(197.0564, grad_fn=<AddBackward0>)
tensor(191.2206, grad_fn=<AddBackward0>)
tensor(185.5962, grad_fn=<AddBackward0>)
tensor(180.1154, grad_fn=<AddBackward0>)
tensor(174.7018, grad_fn=<AddBackward0>)
tensor(169.3667, grad_fn=<AddBackward0>)
tensor(164.0524, grad_fn=<AddBackward0>)
tensor(158.7507, grad_fn=<AddBackward0>)
tensor(153.4765, grad_fn=<AddBackward0>)


In [15]:
# save
# 方法一
torch.save(cnn, './models/nn_loss_optimize.pth')
# 方法二 推薦
torch.save(cnn.state_dict(), "./models/nn_loss_optimize2.pth")

In [17]:
# 方法一加载model要确保有model的类 例如CNN()类
model = torch.load("./models/nn_loss_optimize.pth", weights_only=False)
print(model)
model2 = torch.load("./models/nn_loss_optimize2.pth", weights_only=True)
print(model2)
cnn2 = CNN()
cnn2.load_state_dict(model2)
print(cnn2)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)
OrderedDict([('conv1.weight', tensor([[[[-0.0787,  0.0056,  0.0731,  0.0079, -0.0323],
          [ 0.0100,  0.0163,  0.1113, -0.0348, -0.1175],
          [-0.0412, -0.0196,  0.0605, -0.1039, -0.1358],
          [-0.0200,  0.0535,  0.0058, -0.0088,  0.0790],
          [ 0.0408, -0.1384, -0.0654,  0.0932,  0.0206]],

         [[-

![loss](./LossFunction.png)